# Evaluate Nearby Words in String
This notebook takes a file containing a string, evaluates segments between start/stop words (ending when reaching a "finish" word), and then provides statistics on proximity of words to each other.

## Setup Global Variables


In [1]:
# the file to parse
file = 'mobydick_ch1-21.txt'
# start and stop words - use lower-case as all words are forced to lower-case
start_word = 'ishmael'
stop_word = 'affghanistan'
# word to finish on
finish_word = 'meningitis'
# +/- variance to evaluate
word_range = 5
# word to reference
word = 'a'
# word-position to evaluate (eg., 1 or -2) -- must be within "word_range" above
before_after = 1
# where to find the database
db_file = "pythonsqlite3.db"

## Helper functions

Make a helper-function ready for printing out the DB results:

In [2]:
def print_stats(arr, word, before_after):
    
    
    def add_buffer(word, desired):
        buf = ''
        for i in range(desired-len(word)):
            buf += ' '
        return(f'{word}{buf}')
        
    
    print(f"""
        [For {before_after} {'before' if before_after < 0 else 'after'} "{word}", expect word:]\
        [% of the time]\
        [count]\
    """)
    for item in arr:
        print(f"""
        {add_buffer(item[1], 35)}\
        {add_buffer(str(round(item[3] * 100, 2)), 15)}\
        {add_buffer(str(item[2]), 12)}\
        """)
        
print_stats([('foo', 'bar', 5, 0.25),('foo', 'bar2', 1, .500000001)], 'foo', -1)


        [For -1 before "foo", expect word:]        [% of the time]        [count]    

        bar                                        25.0                   5                   

        bar2                                       50.0                   1                   


## Setup and manage database

In [3]:
import sqlite3
from sqlite3 import Error
db_file = db_file if 'db_file' in locals() else "pythonsqlite3.db"
sql_create_table = """ CREATE TABLE IF NOT EXISTS words (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        refword STRING,
                        word STRING
                    ); """

# Setup connection function
def create_connection(db_file):
    # create a connection to db
    try:
        conn = sqlite3.connect(db_file)
        print("Connected", sqlite3.version)
    except Error as e:
        print(e)
    return conn

# Setup closing function
def close_connection(conn):
    conn.close()
    print('Connection closed')
        

# Setup create-table function
def create_table(conn):
    try:
        c = conn.cursor()
        c.execute(sql_create_table)
        print('Table created or already exists')
    except Error as e:
        print(e)
        
def insert(conn, word_pairs):
    try:
        c = conn.cursor()
        this_sql = f'INSERT INTO words(refword, word) VALUES (?, ?);'
        c.executemany(this_sql, word_pairs)
        print('Inserted')
    except Error as e:
        print(e)
        
        
def select_from_table(conn):
    try:
        c = conn.cursor()
        c.execute("SELECT * from words")
    except Error as e:
        print(e)
    rows = c.fetchall()
    return rows


def select_stats(conn, by_word):
    print('by word', by_word)
    try:
        c = conn.cursor()
        c.execute(f"""
            SELECT refword, word, COUNT(1) AS total, (COUNT(1) / (t.cnt * 1.0)) AS percent
            FROM words
            CROSS JOIN (SELECT COUNT(1) AS cnt FROM words WHERE refword='{by_word}' GROUP BY refword) t
            WHERE refword='{by_word}'
            GROUP BY refword, word
            ORDER BY total DESC
        """)
    except Error as e:
        print(e)
    rows = c.fetchall()
    return rows


def delete(conn):
    try:
        c = conn.cursor()
        c.execute("DELETE FROM words WHERE id=id")
        print('Deleted everything in the words table')
    except Error as e:
        print(e)

        
conn = create_connection(db_file)

if conn is not None:
    # create table
    create_table(conn)
    # insert sample data
    insert(conn, [['foo', 'bar'], ['foo', 'bar'], ['foo', 'bar2'], ['foo2', 'bar2']])
    # read the table
    print("Contents of words table", select_from_table(conn))
    print('Stats: ', select_stats(conn, 'foo'))
    # delete the test
    delete(conn)
    # close connection
    close_connection(conn)
else:
    print('NO CONNECTION EXISTS')


Connected 2.6.0
Table created or already exists
Inserted
Contents of words table [(1, 'foo', 'bar'), (2, 'foo', 'bar'), (3, 'foo', 'bar2'), (4, 'foo2', 'bar2')]
by word foo
Stats:  [('foo', 'bar', 2, 0.6666666666666666), ('foo', 'bar2', 1, 0.3333333333333333)]
Deleted everything in the words table
Connection closed


Great, our database works and now has a table

## Get File
Ok, cool, now we'll get the file

In [4]:
# Punctuation to strip out
to_delete = ['\ufeff', '.', ',', '!', ';', '?', '“', '”']
to_replace = ['—', '\n', '\t']

def get_file(file):
    try:
        textFile = open(file, encoding='utf8')
    except Error as e:
        print(e)
    return textFile.read()
def parse_and_shatter(text):
    text = text.lower()
    for item in to_delete:
        text = text.replace(item, '')
    for item in to_replace:
        text = text.replace(item, ' ')
    return text.split()
def filter_by_start_stop(arr, start_word, stop_word, finish_word):
    rtn = []
    recording = False
    for word in arr:
        # If reach finish word, stop completely (separated from above in case finish_word is stop_word)
        if word == finish_word:
            break
        
        # If reach starting word, begin recording words
        if word == start_word:
            recording = True
            
        # If reach stopping word, stop recording
        if word == stop_word:
            recording = False
        
        # If not a key-word and recording
        if word != start_word and word != stop_word and recording:
            rtn.append(word)
    return rtn
    

text = get_file('mobydick_ch1.txt')
arr = parse_and_shatter(text)
print(filter_by_start_stop(arr, 'chapter', 'precisely', 'precisely'))


['1', 'loomings', 'call', 'me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long']


## Group to word-pairs
Ok, so we got our file, now to group it into word-pairs:

In [5]:
word_range = word_range if 'word_range' in locals() else 2
def group_to_word_pairs(arr):
    i = 0
    pairs = []
    for word in arr:
        for compare_i in range(-word_range, word_range + 1):
            c_i = i + compare_i
            if compare_i != 0 and c_i >= 0 and c_i < len(arr):
                pairs.append([f'{word}{compare_i}', arr[c_i]])
        i += 1
    return pairs
print('Word pairs:', group_to_word_pairs(['foo', 'bar', 'dev', 'ops', 'jupyter']))

Word pairs: [['foo1', 'bar'], ['foo2', 'dev'], ['foo3', 'ops'], ['foo4', 'jupyter'], ['bar-1', 'foo'], ['bar1', 'dev'], ['bar2', 'ops'], ['bar3', 'jupyter'], ['dev-2', 'foo'], ['dev-1', 'bar'], ['dev1', 'ops'], ['dev2', 'jupyter'], ['ops-3', 'foo'], ['ops-2', 'bar'], ['ops-1', 'dev'], ['ops1', 'jupyter'], ['jupyter-4', 'foo'], ['jupyter-3', 'bar'], ['jupyter-2', 'dev'], ['jupyter-1', 'ops']]


## Push to Database
Ok, now that all the needed functions are ready, go ahead and run it all:

In [6]:
db_file = db_file if 'db_file' in locals() else "pythonsqlite3.db"
word = word if 'word' in locals() else 'and'
before_after = before_after if 'before_after' in locals() else 1
start_word = start_word if 'start_word' in locals() else 'and'
stop_word = stop_word if 'stop_word' in locals() else 'meningitis'
finish_word = finish_word if 'finish_word' in locals() else 'meningitis'
file = file if 'file' in locals() else 'mobydick_ch1-21'

def fill_db():
    # Prep the word-pairs
    file_contents = get_file(file)
    arr = parse_and_shatter(file_contents)
    arr = filter_by_start_stop(arr, start_word, stop_word, finish_word)
    pairs = group_to_word_pairs(arr)
    conn = create_connection(db_file)

    if conn is not None:
        # create table
        create_table(conn)
        # insert sample data
        insert(conn, pairs)
        stats = select_stats(conn, f'{word}{before_after}')
        # read the table
        print('STATS:')
        print_stats(stats, word, before_after)
        # delete the test
        delete(conn)
        # close connection
        close_connection(conn)
    else:
        close_connection(conn)
        print('Connection closed')
fill_db()

Connected 2.6.0
Table created or already exists
Inserted
by word a1
STATS:

        [For 1 after "a", expect word:]        [% of the time]        [count]    

        little                                     3.84                   40                  

        good                                       2.3                    24                  

        man                                        1.63                   17                  

        ship                                       1.25                   13                  

        great                                      1.15                   12                  

        sort                                       1.15                   12                  

        very                                       1.15                   12                  

        few                                        1.06                   11                  

        long                                       1.06                  

        cape-horner                                0.1                    1                   

        captain                                    0.1                    1                   

        careless                                   0.1                    1                   

        carpenter’s                                0.1                    1                   

        carved                                     0.1                    1                   

        castaway                                   0.1                    1                   

        cataract                                   0.1                    1                   

        celebrated                                 0.1                    1                   

        century                                    0.1                    1                   

        cheating                                   0.1                    1                   

        cheek                           

Deleted everything in the words table
Connection closed
